# **Proyecto Semestral - MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**


**Integrantes:**

- **Cristian Oyarzo M.**
- **Sebastián Quenti A.**


## **Configuración e Importación de Librerías**

En esta sección, se importarán las librerías necesarias para el desarrollo y análisis del proyecto.


In [202]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_curve, auc, make_scorer
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_recall_curve, auc, make_scorer
from sklearn.metrics import classification_report

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin

from zipfile import ZipFile
import os
import joblib

## Introducción

El proyecto se centra en el desarrollo de un modelo predictivo capaz de estimar la **probabilidad de morosidad** de los clientes de un banco. El objetivo principal es identificar aquellos clientes que son más propensos a incumplir en sus pagos, permitiendo a la institución financiera tomar decisiones informadas para gestionar riesgos y maximizar ingresos.

El conjunto de datos incluye variables relacionadas con el perfil financiero y comportamental de los clientes. Estas variables abarcan tanto datos numéricos (por ejemplo, ingresos, montos adeudados) como categóricos (por ejemplo, historial crediticio). El problema presenta un marcado desbalance entre las clases, ya que la cantidad de clientes morosos es significativamente menor en comparación con los clientes no morosos.

Dada la naturaleza desbalanceada de los datos, se seleccionó el **Área Bajo la Curva de Precisión-Recall (AUC-PR)** como métrica principal. Esta métrica es adecuada porque se centra en el rendimiento de la clase positiva (clientes morosos) y mide la capacidad del modelo para identificar correctamente a los clientes de alto riesgo mientras minimiza los falsos positivos. El uso de esta métrica es preferible frente a métricas como la precisión global (`accuracy`), que podría ser engañosa en un problema desbalanceado.

Para resolver el problema, se utilizaron cuatro modelos supervisados:
1. **XGBoost**: Un modelo basado en boosting que mostró ser el mejor entre los probados.
2. **CatBoost**: Ideal para manejar datos categóricos de forma nativa.
3. **LightGBM**: Un modelo eficiente y rápido para conjuntos de datos grandes.
4. **Regresión Logística**: Utilizada como referencia base para el análisis comparativo.

Antes del entrenamiento de los modelos, se realizaron transformaciones intermedias mediante **pipelines** y **ColumnTransformers**, que incluyeron:
- Escalado de variables numéricas.
- Codificación de variables categóricas.
- Manejo de valores faltantes, cuando fue necesario.


## **Etapa 1**

Se cargan los datos proporcionados para esta etapa del proyecto

In [90]:
df_x0 = pd.read_parquet('X_t0.parquet')
df_x1 = pd.read_parquet('X_t1.parquet')
df_y0 = pd.read_parquet('y_t0.parquet')

Para el Análisis Exploratorio de Datos (EDA) se utilizó la herramienta de generación automática de perfiles con el siguiente código:

In [8]:
profile = ProfileReport(
    df_x0,
    title="Análisis Exploratorio de Datos Automático",
    explorative=True
)

# Guarda el informe como un archivo HTML
profile.to_file("EDA_Reporte.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Debido a la altísima cantidad de variables en los datos, esta herramienta permite generar un informe detallado que incluye estadísticas descriptivas, detección de valores faltantes, correlaciones, distribuciones y otros aspectos clave de los datos, facilitando una comprensión global y eficiente de la información disponible.

Se fija una semilla aleatoria para garantizar la reproducibilidad de los resultados a lo largo del desarrollo del proyecto:

In [3]:
rd = 1323

### **Preprocesamiento de Datos**

En esta etapa, se preparan los datos para el modelado mediante la creación de **pipelines** y el uso de **ColumnTransformers**.


In [203]:
# Transformador personalizado para eliminar columnas específicas
class DropColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X.drop(columns=self.columns_to_drop, errors='ignore')

# Especificar las columnas categóricas y numéricas
categorical_features = ['market_ht_trendmode', 'unique_borrow_protocol_count', 'unique_lending_protocol_count']
numerical_features = [col for col in df_x0.columns if col not in categorical_features + ['wallet_address']]

# Pipeline para las variables numéricas
numerical_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())  # Min-Max Scaler
])

# Aplicar ColumnTransformer para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),  # Aplicar transformación a numéricas
        ('cat', 'passthrough', categorical_features)      # No hacer nada a las categóricas
    ]
)
preprocessor.set_output(transform="pandas")  # Forzar salida como DataFrame

# Pipeline general con eliminación de la columna 'wallet_address'
full_pipeline = Pipeline(steps=[
    ('drop_column', DropColumnTransformer(columns_to_drop=['wallet_address'])),  # Eliminar 'wallet_address'
    ('preprocessor', preprocessor)          # Preprocesamiento por columnas
])

In [204]:
df_x0.shape

(44296, 77)

In [205]:
x0_preprocesed = full_pipeline.fit_transform(df_x0)

In [207]:
x0_preprocesed

,num__borrow_block_number,num__borrow_timestamp,num__first_tx_timestamp,num__last_tx_timestamp,num__wallet_age,num__incoming_tx_count,num__outgoing_tx_count,num__net_incoming_tx_count,num__total_gas_paid_eth,num__avg_gas_paid_per_tx_eth,...,num__market_max_drawdown_365d,num__market_natr,num__market_plus_di,num__market_plus_dm,num__market_ppo,num__market_rocp,num__market_rocr,cat__market_ht_trendmode,cat__unique_borrow_protocol_count,cat__unique_lending_protocol_count
0,0.000000,0.000000,0.639080,0.000000,1.296240e-01,0.002005,0.006161,0.408661,0.000785,0.002363,...,0.000000,0.163508,0.533767,0.271149,0.552217,0.601832,0.601832,1,0,1
1,0.000002,0.000004,0.639080,0.000007,1.296249e-01,0.002015,0.006176,0.408661,0.000788,0.002367,...,0.000000,0.163508,0.533767,0.271149,0.552217,0.601832,0.601832,1,1,1
2,0.000003,0.000005,0.639080,0.000010,1.296252e-01,0.002025,0.006190,0.408661,0.000792,0.002372,...,0.000000,0.163508,0.533767,0.271149,0.552217,0.601832,0.601832,1,1,1
3,0.000211,0.000212,0.768032,0.000013,8.946450e-05,0.000071,0.000788,0.409813,0.001193,0.025979,...,0.000000,0.163508,0.533767,0.271149,0.552217,0.601832,0.601832,1,0,1
4,0.000554,0.000554,0.768199,0.000558,8.242286e-07,0.000010,0.000042,0.410096,0.000006,0.002415,...,0.000000,0.163508,0.533767,0.271149,0.552217,0.601832,0.601832,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44291,0.999989,0.999990,0.997972,0.998857,2.044457e-03,0.000496,0.000746,0.410084,0.000553,0.012725,...,0.481678,0.152486,0.262820,0.184840,0.566551,0.511837,0.511837,0,1,1
44292,0.999997,0.999997,0.978759,0.998863,2.135308e-02,0.005418,0.008525,0.409680,0.011308,0.022749,...,0.481678,0.152486,0.262820,0.184840,0.566551,0.511837,0.511837,0,1,1
44293,0.999998,0.999999,0.839856,0.998866,1.609377e-01,0.003970,0.005950,0.409921,0.004629,0.013343,...,0.481678,0.152486,0.262820,0.184840,0.566551,0.511837,0.511837,0,1,1
44294,1.000000,1.000000,0.796884,0.998851,2.041205e-01,0.000446,0.000689,0.410078,0.000654,0.016621,...,0.481678,0.152486,0.262820,0.184840,0.566551,0.511837,0.511837,0,1,1


Se divide el conjunto de datos en tres partes para garantizar una evaluación adecuada del modelo:

- **70%** para entrenamiento: Utilizado para ajustar el modelo.
- **15%** para validación: Usado para evaluar los modelos generados por el proceso de búsqueda de hiperparámetros con `GridSearchCV`.
- **15%** para prueba: Reservado para evaluar el rendimiento final del mejor modelo encontrado.


In [5]:
# Dividir en entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=rd, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=rd, stratify=y_temp)

### **Inicio de la Etapa de Modelado**

Se comienza la construcción y evaluación de modelos predictivos con el objetivo de estimar la probabilidad de morosidad de los clientes.


Métrica utilizada:

In [11]:
# Definir una métrica personalizada para AUC-PR
def custom_auc_pr(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Crear un scorer basado en la métrica personalizada
auc_pr_scorer = make_scorer(custom_auc_pr, response_method = 'predict_proba')

#### **Modelo Base: Regresión Logística**

Se implementa un modelo de Regresión Logística como referencia base para el análisis comparativo.


In [ ]:
# Definir un modelo base
log_reg = LogisticRegression(random_state=rd, solver='liblinear')

# Definir un rango reducido de hiperparámetros
param_grid = {
    'C': [0.01, 0.1, 1, 10],  # Inverso de la regularización
    'penalty': ['l1', 'l2']   # Tipo de regularización
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV usando el conjunto de entrenamiento
grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search con el conjunto de entrenamiento
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y el mejor puntaje de la validación cruzada
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")

# Evaluar el modelo con el conjunto de validación
best_model_lr = grid_search.best_estimator_
y_val_pred = best_model_lr.predict_proba(X_val)[:, 1]  # Obtener las probabilidades predichas para la clase positiva
val_auc_pr = custom_auc_pr(y_val, y_val_pred)  # Calcular AUC-PR en el conjunto de validación
print(f"AUC-PR en el conjunto de validación: {val_auc_pr}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores parámetros: {'C': 1, 'penalty': 'l1'}
Mejor puntuación de validación cruzada (AUC-PR): 0.864619698206545
AUC-PR en el conjunto de validación: 0.8578848730560438


c:\Users\crist\anaconda3\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### **Modelo 1: XGBoost**

Se utiliza el clasificador `XGBClassifier` de la librería `xgboost` para construir el primer modelo predictivo. Este modelo es conocido por su capacidad de manejar grandes volúmenes de datos y características con alta eficiencia, siendo una opción robusta para problemas de clasificación como el planteado en este proyecto.

In [7]:
# Definir modelo
xgb = XGBClassifier(eval_metric='logloss', random_state=rd, scale_pos_weight = 1)

# Definir un rango reducido de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'reg_alpha': [0, 0.1, 1]  
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")
# Evaluar el modelo con el conjunto de validación
best_model_xgb = grid_search.best_estimator_
y_val_pred = best_model_xgb.predict_proba(X_val)[:, 1]  # Obtener las probabilidades predichas para la clase positiva
val_auc_pr = custom_auc_pr(y_val, y_val_pred)  # Calcular AUC-PR en el conjunto de validación
print(f"AUC-PR en el conjunto de validación: {val_auc_pr}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 1}
Mejor puntuación de validación cruzada (AUC-PR): 0.9433971825531667
AUC-PR en el conjunto de validación: 0.9435305574148607


#### **Modelo 2: CatBoost**

Se utiliza el modelo de clasificación `CatBoostClassifier`.

In [8]:
# Definir un modelo base
catboost = CatBoostClassifier(
    verbose=0,
    random_seed=rd
)

# Definir un rango reducido de hiperparámetros
param_grid = {
    'iterations': [100, 200],
    'depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'l2_leaf_reg': [1, 3, 5] 
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=catboost,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")    
# Evaluar el modelo con el conjunto de validación
best_model_cat = grid_search.best_estimator_
y_val_pred = best_model_cat.predict_proba(X_val)[:, 1]  # Obtener las probabilidades predichas para la clase positiva
val_auc_pr = custom_auc_pr(y_val, y_val_pred)  # Calcular AUC-PR en el conjunto de validación
print(f"AUC-PR en el conjunto de validación: {val_auc_pr}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\crist\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\crist\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\crist\anaconda3\lib\site-packages\catboost\core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Users\crist\anaconda3\lib\site-packages\catboost\core.py", line 2

Mejores parámetros: {'depth': 5, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Mejor puntuación de validación cruzada (AUC-PR): 0.9302120755133145
AUC-PR en el conjunto de validación: 0.9281045158613274


#### **Modelo 3: LightGBM**

Se utiliza el modelo de clasificación `LGBMClassifier`.


In [9]:
# Definir un modelo base
lightgbm = LGBMClassifier(random_state=rd)

# Definir un rango reducido de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],  
    'max_depth': [3, 5],       
    'learning_rate': [0.01, 0.1],
    'reg_alpha': [0, 0.1]    
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV usando el conjunto de entrenamiento
grid_search = GridSearchCV(
    estimator=lightgbm,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)
    
# Ejecutar el grid search con el conjunto de entrenamiento
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y el mejor puntaje de la validación cruzada
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")

# Evaluar el modelo con el conjunto de validación
best_model_l = grid_search.best_estimator_
y_val_pred = best_model_l.predict_proba(X_val)[:, 1]  # Obtener las probabilidades predichas para la clase positiva
val_auc_pr = custom_auc_pr(y_val, y_val_pred)  # Calcular AUC-PR en el conjunto de validación
print(f"AUC-PR en el conjunto de validación: {val_auc_pr}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[LightGBM] [Info] Number of positive: 15859, number of negative: 15148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014085 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17127
[LightGBM] [Info] Number of data points in the train set: 31007, number of used features: 76
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511465 -> initscore=0.045869
[LightGBM] [Info] Start training from score 0.045869
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

#### **Evaluación de Modelos en el Conjunto de Prueba**

En esta sección, habiendo iterado las grillas en todos los modelos, se evalúan los cuatro modelos construidos previamente (XGBoost, CatBoost, LightGBM y Regresión Logística) en el conjunto de prueba para medir su rendimiento final. Para cada modelo, se calcula la métrica principal del proyecto, el **AUC-PR**, y se comparan los resultados para identificar el modelo con mejor desempeño.


In [10]:
# Evaluación en el conjunto de prueba
def evaluate_model_on_test(model, model_name, X_test, y_test):
    # Obtener probabilidades predichas para la clase positiva
    y_test_pred = model.predict_proba(X_test)[:, 1]
    
    # Calcular AUC-PR
    test_auc_pr = custom_auc_pr(y_test, y_test_pred)
    print(f"{model_name} - AUC-PR en el conjunto de prueba: {test_auc_pr:.4f}")
    
    # Reporte de clasificación
    y_test_pred_class = model.predict(X_test)
    print(f"Reporte de Clasificación para {model_name}:\n")
    print(classification_report(y_test, y_test_pred_class))
    print("-" * 50)
    return test_auc_pr

# Evaluar los cuatro modelos
print("Evaluación en el conjunto de prueba:")
results = {}

# Evaluar Regresión Logística
results['Logistic Regression'] = evaluate_model_on_test(best_model_lr, "Logistic Regression", X_test, y_test)

# Evaluar XGBoost
results['XGBoost'] = evaluate_model_on_test(best_model_xgb, "XGBoost", X_test, y_test)

# Evaluar CatBoost
results['CatBoost'] = evaluate_model_on_test(best_model_cat, "CatBoost", X_test, y_test)

# Evaluar LightGBM
results['LightGBM'] = evaluate_model_on_test(best_model_l, "LightGBM", X_test, y_test)


# Comparación final de resultados
print("Resultados finales:")
for model, auc_pr in results.items():
    print(f"{model}: AUC-PR = {auc_pr:.4f}")


Evaluación en el conjunto de prueba:
Logistic Regression - AUC-PR en el conjunto de prueba: 0.8645
Reporte de Clasificación para Logistic Regression:

              precision    recall  f1-score   support

           0       0.73      0.78      0.75      3246
           1       0.78      0.72      0.74      3399

    accuracy                           0.75      6645
   macro avg       0.75      0.75      0.75      6645
weighted avg       0.75      0.75      0.75      6645

--------------------------------------------------
XGBoost - AUC-PR en el conjunto de prueba: 0.9437
Reporte de Clasificación para XGBoost:

              precision    recall  f1-score   support

           0       0.81      0.91      0.86      3246
           1       0.90      0.80      0.85      3399

    accuracy                           0.85      6645
   macro avg       0.86      0.85      0.85      6645
weighted avg       0.86      0.85      0.85      6645

--------------------------------------------------
Cat

#### **Entrega Parcial 1**

Dado que el modelo **XGBoost** obtuvo el mejor desempeño con un **AUC-PR de 0.9437**, se decide seleccionarlo como el modelo final para realizar una búsqueda más exhaustiva de hiperparámetros con una grilla ampliada. Esto permitirá explorar un espacio más amplio de configuraciones y, potencialmente, mejorar aún más su desempeño.




Para aprovechar al máximo el potencial del modelo **XGBoost**, se realiza una búsqueda ampliada de hiperparámetros utilizando **GridSearchCV**. En esta búsqueda:

- Se utiliza la **totalidad de los datos de entrenamiento, validación y testeo combinados** para maximizar el uso de la información disponible y obtener una configuración más robusta.
- La validación cruzada se realiza con **StratifiedKFold** (5 particiones), asegurando un equilibrio en las clases durante la evaluación.

La nueva grilla de hiperparámetros incluye:

- **`n_estimators`**: Número de árboles en el modelo, con valores entre 50 y 500.
- **`max_depth`**: Profundidad máxima de los árboles, con valores de 3 a 10, para controlar la complejidad.
- **`learning_rate`**: Tasa de aprendizaje, probando desde tasas bajas (0.001) hasta más agresivas (0.2).
- **`reg_alpha` y `reg_lambda`**: Parámetros de regularización L1 y L2, explorando la influencia de la penalización en el ajuste.
- **`subsample`**: Proporción de datos utilizados para construir cada árbol, con valores de 0.5 y 1.


El uso de una grilla ampliada y de validación cruzada es crucial para garantizar que el modelo seleccionado no solo se ajuste bien a los datos disponibles, sino que también generalice de manera efectiva a nuevos datos. Este enfoque exhaustivo asegura que se exploren configuraciones más complejas, maximizando el desempeño del modelo en la tarea de predecir la probabilidad de morosidad.

Cabe destacar que para la entrega parcial 1, en la etapa 1 en general, no se realizó el preprocesamiento de datos presente en el informe, debido a la carga académica de los integrantes del grupo y las características del modelo de resistencia a la escala de los datos y alta dimensionalidad.

In [ ]:
# Cargar los datos
df_x0 = pd.read_parquet('X_t0.parquet')
df_y0 = pd.read_parquet('y_t0.parquet')

# Asegurarnos de que la variable objetivo sea un vector
y = df_y0.values.ravel()

# Eliminar la columna de texto
text_column = 'wallet_address'
X = df_x0.drop(columns=[text_column])

# Definir una métrica personalizada para AUC-PR
def custom_auc_pr(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Crear un scorer basado en la métrica personalizada
auc_pr_scorer = make_scorer(custom_auc_pr, response_method = 'predict_proba')

# Definir un modelo base
xgb = XGBClassifier(eval_metric='logloss', random_state=rd, scale_pos_weight = 1)

# Definir una grilla ampliada de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 1, 10],
    'reg_lambda': [1, 5, 10],
    'subsample': [0.5, 1]
}


# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X, y)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")

Tras realizar la búsqueda exhaustiva de hiperparámetros mediante `GridSearchCV`, los mejores parámetros encontrados para el modelo **XGBoost** fueron los siguientes:

- **`learning_rate`:** 0.2  
- **`max_depth`:** 10  
- **`n_estimators`:** 500  
- **`reg_alpha`:** 0  
- **`reg_lambda`:** 1  
- **`subsample`:** 1  


In [ ]:
# Obtener el mejor modelo de grid search
best_model = grid_search.best_estimator_
X1 = df_x1.drop(columns=[text_column])

def generateFiles(predict_data, clf_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    ---------------
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf

    Ouput
    ---------------
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict_proba(predict_data)[:, 1]
    with open('./predictions.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)
    
    with ZipFile('predictions.zip', 'w') as zipObj:
        zipObj.write('predictions.txt')
    os.remove('predictions.txt')

generateFiles(X1, best_model)

In [ ]:
# Guardar el mejor modelo
joblib.dump(best_model, 'best_model1.pkl')

print("El mejor modelo ha sido guardado en 'best_model1.pkl'")

##### Conclusión de la etapa

El modelo entregado en CodaLab entregó un **AUC-PR de 0.7914172453**, donde, si bien se obtuvo un buen resultado, no se hicieorn los procedimientos esperados para la entrega que podrían haber mejorado sustancialmente el modelo de predicción. Particularmente, utilizar el preprocesamiento, mejorar la capacidad de generalización del modelo y probar una mayor variedad de modelos e hiperparámetros.

## Etapa 2

In [210]:
df_x0 = pd.read_parquet('X_t0.parquet')
df_x1_v2 = pd.read_parquet('X_t1_v2.parquet')
df_x2 = pd.read_parquet('X_t2.parquet')

df_y0 = pd.read_parquet('y_t0.parquet')
df_y1 = pd.read_parquet('y_t1.parquet')

In [212]:
# Unir los datasets de X e Y
df_x_combined = pd.concat([df_x0, df_x1_v2], ignore_index=True)
df_y_combined = pd.concat([df_y0, df_y1], ignore_index=True)

# Aplicar el pipeline al conjunto combinado
X_preprocessed = full_pipeline.fit_transform(df_x_combined)

# Configurar y entrenar el modelo XGBoost
xgb_model = XGBClassifier(
    learning_rate=0.2,
    max_depth=10,
    n_estimators=500,
    reg_alpha=0,
    reg_lambda=1,
    subsample=1,
    random_state=rd,
    eval_metric='logloss'
)

xgb_model.fit(X_preprocessed, df_y_combined)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=1323, ...)

In [214]:
def generateFiles(predict_data, clf_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    ---------------
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf

    Ouput
    ---------------
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict_proba(predict_data)[:, 1]
    with open('./predictions.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)
    
    with ZipFile('predictions.zip', 'w') as zipObj:
        zipObj.write('predictions.txt')
    os.remove('predictions.txt')
    
#  Preprocesar X2 y generar predicciones
X2_preprocessed = full_pipeline.transform(df_x2)
generateFiles(X2_preprocessed, xgb_model)